In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class LeftoverCoarseLabelFinal(dj.Computed):
    definition = """
    # Ensure that Vertex labels for pinky.LeftoverProofreadLabel match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.LeftoverProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.LeftoverProofreadLabel
    """
    
    key_source = pinky.LeftoverProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.LeftoverProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.ExcitatoryLeftoverMeshes & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [4]:
start = time.time()
LeftoverCoarseLabelFinal.populate(reserve_jobs=True)
print(time.time() - start)

648518346349502049
Segment 648518346349502049 vertex labels regenerated in: 0.2048041820526123 seconds.
648518346349502920
Segment 648518346349502920 vertex labels regenerated in: 0.28079843521118164 seconds.
648518346349503086
Segment 648518346349503086 vertex labels regenerated in: 0.22464489936828613 seconds.
648518346349503453
Segment 648518346349503453 vertex labels regenerated in: 0.3115725517272949 seconds.
648518346349503643
Segment 648518346349503643 vertex labels regenerated in: 0.2004847526550293 seconds.
648518346349504065
Segment 648518346349504065 vertex labels regenerated in: 0.23406577110290527 seconds.
648518346349504324
Segment 648518346349504324 vertex labels regenerated in: 0.3227880001068115 seconds.
648518346349504868
Segment 648518346349504868 vertex labels regenerated in: 0.27727699279785156 seconds.
648518346349505250
Segment 648518346349505250 vertex labels regenerated in: 0.3200502395629883 seconds.
648518346349505640
Segment 648518346349505640 vertex labels 

In [5]:
#(schema.jobs & "table_name='__coarse_label_final'").delete()